# Spam Sms Classifier

##### Required depedencies

In [32]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, Tokenizer, StopWordsRemover, CountVectorizer, IDF, VectorAssembler
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier, RandomForestClassifier, NaiveBayes
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

## Load data and inspect it

In [33]:
spark = SparkSession.builder.appName("SMS Spam Classifier").getOrCreate()

import logging
logger = spark._jvm.org.apache.log4j
logging.getLogger("py4j").setLevel(logging.ERROR)
logger.LogManager.getLogger("org").setLevel(logger.Level.ERROR)

##### Nous pouvons maintenant charger les données dans un DataFrame Spark et renommer les colonnes :

In [34]:
data = spark.read.format("csv").option("header", "false").option("delimiter", "\t").load("sms_spam_collection.txt")
data = data.withColumnRenamed("_c0", "class").withColumnRenamed("_c1", "text")

In [35]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



## trasform data

In [36]:
# Nous convertissons ensuite la colonne "class" en valeurs numériques
indexer = StringIndexer(inputCol="class", outputCol="label")
data = indexer.fit(data).transform(data)

In [37]:
# convertis la colonne de texte en une liste de mots sans les mots vides
tokenizer = Tokenizer(inputCol="text", outputCol="words")
stopwords_remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
data = tokenizer.transform(data)
data = stopwords_remover.transform(data)

In [38]:
# transforme les listes de mots en vecteurs d'occurrences de mots
count_vectorizer = CountVectorizer(inputCol="filtered_words", outputCol="raw_features")
model = count_vectorizer.fit(data)
data = model.transform(data)

In [39]:
idf = IDF(inputCol="raw_features", outputCol="features")
idf_model = idf.fit(data)
data = idf_model.transform(data)

In [40]:
assembler = VectorAssembler(inputCols=["features"], outputCol="features_vector")
data = assembler.transform(data)

In [41]:
data.describe().show()

+-------+-----+--------------------+------------------+
|summary|class|                text|             label|
+-------+-----+--------------------+------------------+
|  count| 5574|                5574|              5574|
|   mean| null|               645.0|0.1340150699677072|
| stddev| null|                null|0.3406990688361999|
|    min|  ham| &lt;#&gt;  in mc...|               0.0|
|    max| spam|… we r stayin her...|               1.0|
+-------+-----+--------------------+------------------+



In [42]:
(training_data, test_data) = data.randomSplit([0.7, 0.3], seed=100)

In [43]:
training_data.describe().show()

23/02/27 14:16:38 WARN DAGScheduler: Broadcasting large task binary with size 1192.5 KiB
23/02/27 14:16:38 WARN DAGScheduler: Broadcasting large task binary with size 1176.9 KiB
+-------+-----+--------------------+------------------+
|summary|class|                text|             label|
+-------+-----+--------------------+------------------+
|  count| 3907|                3907|              3907|
|   mean| null|               645.0| 0.135398003583312|
| stddev| null|                null|0.3421919853904309|
|    min|  ham| &lt;#&gt;  in mc...|               0.0|
|    max| spam|… we r stayin her...|               1.0|
+-------+-----+--------------------+------------------+



In [44]:
test_data.describe().show()

23/02/27 14:16:39 WARN DAGScheduler: Broadcasting large task binary with size 1192.5 KiB
23/02/27 14:16:39 WARN DAGScheduler: Broadcasting large task binary with size 1176.9 KiB
+-------+-----+--------------------+------------------+
|summary|class|                text|             label|
+-------+-----+--------------------+------------------+
|  count| 1667|                1667|              1667|
|   mean| null|                null|0.1307738452309538|
| stddev| null|                null|0.3372540246678357|
|    min|  ham| and  picking the...|               0.0|
|    max| spam|Ü thk of wat to e...|               1.0|
+-------+-----+--------------------+------------------+



## use different models

##### Logistic regression, decision tree, naive bayes

In [45]:
# Créer les modèles
log_reg = LogisticRegression(featuresCol="features_vector", labelCol="label", maxIter=10)
dt = DecisionTreeClassifier(featuresCol="features_vector", labelCol="label")
nb = NaiveBayes(featuresCol="features_vector", labelCol="label")

# Entraîner les modèles
log_reg_model = log_reg.fit(training_data)
dt_model = dt.fit(training_data)
nb_model = nb.fit(training_data)

23/02/27 14:16:39 WARN DAGScheduler: Broadcasting large task binary with size 1192.0 KiB
23/02/27 14:16:39 WARN DAGScheduler: Broadcasting large task binary with size 1193.5 KiB
23/02/27 14:16:39 WARN DAGScheduler: Broadcasting large task binary with size 1193.5 KiB
23/02/27 14:16:39 WARN DAGScheduler: Broadcasting large task binary with size 1193.5 KiB
23/02/27 14:16:39 WARN DAGScheduler: Broadcasting large task binary with size 1193.5 KiB
23/02/27 14:16:39 WARN DAGScheduler: Broadcasting large task binary with size 1193.5 KiB
23/02/27 14:16:39 WARN DAGScheduler: Broadcasting large task binary with size 1193.5 KiB
23/02/27 14:16:39 WARN DAGScheduler: Broadcasting large task binary with size 1193.5 KiB
23/02/27 14:16:39 WARN DAGScheduler: Broadcasting large task binary with size 1193.5 KiB
23/02/27 14:16:39 WARN DAGScheduler: Broadcasting large task binary with size 1193.5 KiB
23/02/27 14:16:39 WARN DAGScheduler: Broadcasting large task binary with size 1193.5 KiB
23/02/27 14:16:39 WAR

23/02/27 14:16:44 WARN DAGScheduler: Broadcasting large task binary with size 1506.1 KiB


23/02/27 14:16:44 WARN DAGScheduler: Broadcasting large task binary with size 1506.8 KiB
23/02/27 14:16:45 WARN DAGScheduler: Broadcasting large task binary with size 1507.3 KiB
23/02/27 14:16:45 WARN DAGScheduler: Broadcasting large task binary with size 1508.1 KiB
23/02/27 14:16:45 WARN DAGScheduler: Broadcasting large task binary with size 1509.3 KiB
23/02/27 14:16:45 WARN DAGScheduler: Broadcasting large task binary with size 1192.6 KiB
23/02/27 14:16:45 WARN DAGScheduler: Broadcasting large task binary with size 1163.4 KiB


In [46]:
log_reg_predictions = log_reg_model.transform(test_data)
dt_predictions = dt_model.transform(test_data)
nb_predictions = nb_model.transform(test_data)

In [47]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

In [ ]:
log_reg_accuracy = evaluator.evaluate(log_reg_predictions)
dt_accuracy = evaluator.evaluate(dt_predictions)
nb_accuracy = evaluator.evaluate(nb_predictions)

print("Logistic regression accuracy = %g" % (log_reg_accuracy))
print("Decision tree accuracy = %g" % (dt_accuracy))
print("Naive bayes accuracy = %g" % (nb_accuracy))

23/02/27 14:16:45 WARN DAGScheduler: Broadcasting large task binary with size 1321.1 KiB
23/02/27 14:16:46 WARN DAGScheduler: Broadcasting large task binary with size 1216.4 KiB
23/02/27 14:16:46 WARN DAGScheduler: Broadcasting large task binary with size 1419.8 KiB
Logistic regression accuracy = 0.982004
Decision tree accuracy = 0.925615
Naive bayes accuracy = 0.916617


## Random Forest with CrossValidator

In [49]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10, 20, 30]) \
    .addGrid(rf.maxDepth, [2, 4, 6, 8]) \
    .build()

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

cv = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

rf_model = cv.fit(training_data)

23/02/27 14:16:46 WARN DAGScheduler: Broadcasting large task binary with size 1227.0 KiB
23/02/27 14:16:46 WARN DAGScheduler: Broadcasting large task binary with size 1227.1 KiB
23/02/27 14:16:46 WARN DAGScheduler: Broadcasting large task binary with size 1366.0 KiB


23/02/27 14:16:49 WARN DAGScheduler: Broadcasting large task binary with size 1529.3 KiB
23/02/27 14:16:50 WARN DAGScheduler: Broadcasting large task binary with size 1532.9 KiB
23/02/27 14:16:50 WARN DAGScheduler: Broadcasting large task binary with size 1271.1 KiB
23/02/27 14:16:50 WARN DAGScheduler: Broadcasting large task binary with size 1227.0 KiB
23/02/27 14:16:50 WARN DAGScheduler: Broadcasting large task binary with size 1227.1 KiB
23/02/27 14:16:50 WARN DAGScheduler: Broadcasting large task binary with size 1366.0 KiB


23/02/27 14:16:53 WARN DAGScheduler: Broadcasting large task binary with size 1529.3 KiB
23/02/27 14:16:54 WARN DAGScheduler: Broadcasting large task binary with size 1532.9 KiB
23/02/27 14:16:54 WARN DAGScheduler: Broadcasting large task binary with size 1538.0 KiB
23/02/27 14:16:54 WARN DAGScheduler: Broadcasting large task binary with size 1543.5 KiB
23/02/27 14:16:54 WARN DAGScheduler: Broadcasting large task binary with size 1274.9 KiB
23/02/27 14:16:54 WARN DAGScheduler: Broadcasting large task binary with size 1227.0 KiB
23/02/27 14:16:54 WARN DAGScheduler: Broadcasting large task binary with size 1227.1 KiB
23/02/27 14:16:54 WARN DAGScheduler: Broadcasting large task binary with size 1366.0 KiB


23/02/27 14:16:57 WARN DAGScheduler: Broadcasting large task binary with size 1529.3 KiB
23/02/27 14:16:57 WARN DAGScheduler: Broadcasting large task binary with size 1532.9 KiB
23/02/27 14:16:57 WARN DAGScheduler: Broadcasting large task binary with size 1538.0 KiB
23/02/27 14:16:58 WARN DAGScheduler: Broadcasting large task binary with size 1543.5 KiB
23/02/27 14:16:58 WARN DAGScheduler: Broadcasting large task binary with size 1547.3 KiB
23/02/27 14:16:58 WARN DAGScheduler: Broadcasting large task binary with size 1549.8 KiB
23/02/27 14:16:58 WARN DAGScheduler: Broadcasting large task binary with size 1279.1 KiB
23/02/27 14:16:58 WARN DAGScheduler: Broadcasting large task binary with size 1227.0 KiB
23/02/27 14:16:58 WARN DAGScheduler: Broadcasting large task binary with size 1227.1 KiB
23/02/27 14:16:58 WARN DAGScheduler: Broadcasting large task binary with size 1366.0 KiB


23/02/27 14:17:01 WARN DAGScheduler: Broadcasting large task binary with size 1529.3 KiB
23/02/27 14:17:01 WARN DAGScheduler: Broadcasting large task binary with size 1532.9 KiB
23/02/27 14:17:01 WARN DAGScheduler: Broadcasting large task binary with size 1538.0 KiB
23/02/27 14:17:01 WARN DAGScheduler: Broadcasting large task binary with size 1543.5 KiB
23/02/27 14:17:02 WARN DAGScheduler: Broadcasting large task binary with size 1547.3 KiB
23/02/27 14:17:02 WARN DAGScheduler: Broadcasting large task binary with size 1549.8 KiB
23/02/27 14:17:02 WARN DAGScheduler: Broadcasting large task binary with size 1554.7 KiB
23/02/27 14:17:02 WARN DAGScheduler: Broadcasting large task binary with size 1559.8 KiB
23/02/27 14:17:02 WARN DAGScheduler: Broadcasting large task binary with size 1284.0 KiB
23/02/27 14:17:02 WARN DAGScheduler: Broadcasting large task binary with size 1227.0 KiB
23/02/27 14:17:02 WARN DAGScheduler: Broadcasting large task binary with size 1227.1 KiB
23/02/27 14:17:02 WAR

...

23/02/27 14:20:38 WARN DAGScheduler: Broadcasting large task binary with size 1539.6 KiB
23/02/27 14:20:38 WARN DAGScheduler: Broadcasting large task binary with size 1552.5 KiB
23/02/27 14:20:38 WARN DAGScheduler: Broadcasting large task binary with size 1566.4 KiB
23/02/27 14:20:38 WARN DAGScheduler: Broadcasting large task binary with size 1580.2 KiB
23/02/27 14:20:38 WARN DAGScheduler: Broadcasting large task binary with size 1330.0 KiB
23/02/27 14:20:39 WARN DAGScheduler: Broadcasting large task binary with size 1227.0 KiB
23/02/27 14:20:39 WARN DAGScheduler: Broadcasting large task binary with size 1227.1 KiB
23/02/27 14:20:39 WARN DAGScheduler: Broadcasting large task binary with size 1366.0 KiB


23/02/27 14:20:42 WARN DAGScheduler: Broadcasting large task binary with size 1539.6 KiB
23/02/27 14:20:42 WARN DAGScheduler: Broadcasting large task binary with size 1552.5 KiB
23/02/27 14:20:42 WARN DAGScheduler: Broadcasting large task binary with size 1566.4 KiB
23/02/27 14:20:42 WARN DAGScheduler: Broadcasting large task binary with size 1580.2 KiB
23/02/27 14:20:42 WARN DAGScheduler: Broadcasting large task binary with size 1591.0 KiB
23/02/27 14:20:43 WARN DAGScheduler: Broadcasting large task binary with size 1605.9 KiB
23/02/27 14:20:43 WARN DAGScheduler: Broadcasting large task binary with size 1343.1 KiB
23/02/27 14:20:43 WARN DAGScheduler: Broadcasting large task binary with size 1227.0 KiB
23/02/27 14:20:43 WARN DAGScheduler: Broadcasting large task binary with size 1227.1 KiB
23/02/27 14:20:43 WARN DAGScheduler: Broadcasting large task binary with size 1366.0 KiB


23/02/27 14:20:46 WARN DAGScheduler: Broadcasting large task binary with size 1539.6 KiB
23/02/27 14:20:46 WARN DAGScheduler: Broadcasting large task binary with size 1552.5 KiB
23/02/27 14:20:46 WARN DAGScheduler: Broadcasting large task binary with size 1566.4 KiB
23/02/27 14:20:47 WARN DAGScheduler: Broadcasting large task binary with size 1580.2 KiB
23/02/27 14:20:47 WARN DAGScheduler: Broadcasting large task binary with size 1591.0 KiB
23/02/27 14:20:47 WARN DAGScheduler: Broadcasting large task binary with size 1605.9 KiB
23/02/27 14:20:47 WARN DAGScheduler: Broadcasting large task binary with size 1620.3 KiB
23/02/27 14:20:47 WARN DAGScheduler: Broadcasting large task binary with size 1637.3 KiB
23/02/27 14:20:47 WARN DAGScheduler: Broadcasting large task binary with size 1357.0 KiB
23/02/27 14:20:47 WARN DAGScheduler: Broadcasting large task binary with size 1193.3 KiB
23/02/27 14:20:47 WARN DAGScheduler: Broadcasting large task binary with size 1193.4 KiB
23/02/27 14:20:47 WAR

23/02/27 14:20:51 WARN DAGScheduler: Broadcasting large task binary with size 1513.3 KiB


23/02/27 14:20:52 WARN DAGScheduler: Broadcasting large task binary with size 1516.9 KiB
23/02/27 14:20:52 WARN DAGScheduler: Broadcasting large task binary with size 1521.3 KiB
23/02/27 14:20:52 WARN DAGScheduler: Broadcasting large task binary with size 1526.2 KiB
23/02/27 14:20:52 WARN DAGScheduler: Broadcasting large task binary with size 1533.0 KiB
23/02/27 14:20:52 WARN DAGScheduler: Broadcasting large task binary with size 1536.9 KiB
23/02/27 14:20:52 WARN DAGScheduler: Broadcasting large task binary with size 1540.4 KiB
23/02/27 14:20:52 WARN DAGScheduler: Broadcasting large task binary with size 1544.7 KiB


In [50]:
predictions = rf_model.transform(test_data)

In [53]:
accuracy = evaluator.evaluate(predictions)
print("Accuracy = %g" % (accuracy))

23/02/27 14:21:06 WARN DAGScheduler: Broadcasting large task binary with size 1256.4 KiB
Accuracy = 0.885423


## Compare models

In [62]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

model_names = ["Logistic regression", "Decision tree", "Naive bayes", "Random forest"]
models = [log_reg_model, dt_model, nb_model, rf_model]

# Initialize the evaluators
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
evaluator_acc = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
evaluator_pres = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
evaluator_recall = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")

# Create a list of dictionaries to store the results
results = []
for i, model in enumerate(models):
    # Evaluate the model on the test set
    test_predictions = model.transform(test_data)
    f1_score = evaluator_f1.evaluate(test_predictions)
    accuracy = evaluator_acc.evaluate(test_predictions)
    precision = evaluator_pres.evaluate(test_predictions)
    recall = evaluator_recall.evaluate(test_predictions)
    # Add the results to the list
    results.append({
        "Model": model_names[i],
        "F1 Score": f1_score,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall
    })


23/02/27 14:32:24 WARN DAGScheduler: Broadcasting large task binary with size 1323.2 KiB
23/02/27 14:32:24 WARN DAGScheduler: Broadcasting large task binary with size 1323.2 KiB
23/02/27 14:32:24 WARN DAGScheduler: Broadcasting large task binary with size 1323.2 KiB
23/02/27 14:32:24 WARN DAGScheduler: Broadcasting large task binary with size 1323.2 KiB
23/02/27 14:32:24 WARN DAGScheduler: Broadcasting large task binary with size 1218.5 KiB
23/02/27 14:32:25 WARN DAGScheduler: Broadcasting large task binary with size 1218.5 KiB
23/02/27 14:32:25 WARN DAGScheduler: Broadcasting large task binary with size 1218.5 KiB
23/02/27 14:32:25 WARN DAGScheduler: Broadcasting large task binary with size 1218.5 KiB
23/02/27 14:32:25 WARN DAGScheduler: Broadcasting large task binary with size 1421.9 KiB
23/02/27 14:32:25 WARN DAGScheduler: Broadcasting large task binary with size 1421.9 KiB
23/02/27 14:32:25 WARN DAGScheduler: Broadcasting large task binary with size 1421.9 KiB
23/02/27 14:32:26 WAR

In [63]:
# Print the results table
print("{:<25} {:<10} {:<10} {:<10} {:<10}".format('Model', 'F1 Score', 'Accuracy', 'Precision', 'Recall'))
print("=" * 90)
for result in results:
    print("{:<25} {:<10.2f} {:<10.2f} {:<10.2f} {:<10.2f}".format(result['Model'], result['F1 Score'], result['Accuracy'], result['Precision'], result['Recall']))

Model                     F1 Score   Accuracy   Precision  Recall    
Logistic regression       0.98       0.98       0.98       0.98      
Decision tree             0.92       0.93       0.92       0.93      
Naive bayes               0.92       0.92       0.94       0.92      
Random forest             0.84       0.89       0.90       0.89      


We can notice that the logistic regression model performed the best with an F1 score of 0.98, a precision of 0.98 and a recall of 0.98. This means that the model did very well in classifying SMS as spam or healthy.

The decision tree model also performed well with an F1 score of 0.92, precision of 0.92 and recall of 0.93.

However, the Naive Bayes model performed similarly to the decision tree in terms of F1 score and recall, but had slightly higher precision.

The Random Forest model performed worse than the other models with an F1 score of 0.84, precision of 0.90 and recall of 0.89.

In general, the choice of model will depend on the priorities of the application. If precision is the priority, then the Naive Bayes model may be preferred, but if recall is the priority, the decision tree model may be preferred. However, if balancing precision and recall is the priority, then the logistic regression model may be chosen due to its superior overall performance.